In [5]:
!pip install ckiptagger
!pip install hdbscan
!pip install bertopic

  Using cached hdbscan-0.8.28.tar.gz (5.2 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp39-cp39-win_amd64.whl size=607537 sha256=65a0e6d414354cc389a16cd668925f5cd073d531658b5ea46a4ec153900607fc
  Stored in directory: c:\users\taoxu\appdata\local\pip\cache\wheels\97\2d\1e\d9907e8f806ee949f9effc41004d7f32e862f6f67d9157812d
Successfully built hdbscan
  Using cached bertopic-0.12.0-py2.py3-none-any.whl (90 kB)
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached PyYAML-5.4.1-cp39-cp39-win_amd64.whl (213 kB)
  Using cached umap_learn-0.5.3-py3-none-any.whl
  Using cached transformers-4.23.1-py3-none-any.whl (5.3 MB)
  Using cached h

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.20 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.20 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.
conda-repo-cli 1.0.20 requires PyYAML==6.0, but you have pyyaml 5.4.1 which is incompatible.


In [ ]:
import re
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
from bertopic import BERTopic
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# get the tradition Chinese model
tokenizer = AutoTokenizer.from_pretrained("elliotthwang/t5-small-finetuned-xlsum-chinese-tradition")
model = AutoModelForSeq2SeqLM.from_pretrained("elliotthwang/t5-small-finetuned-xlsum-chinese-tradition")

# get all the file
import os
years = []
for path, dir_list, file_list in os.walk(r"C:\Users\TaoXU\Desktop\Lib_DA\year_txt"):
    for file in file_list:
        year = file
        years.append(year)

base_path = "C:\\Users\\TaoXU\\Desktop\\Lib_DA\\year_txt\\"
for i in years:
    file_path = base_path +i 
    sentences = ""
    with open(file_path, encoding="utf-8") as f:
        for line in f.readlines():
            line = re.sub('\n', '', line)  # 去掉列表中每一个元素的换行符
            line = re.sub('[a-zA-Z0-9]', '', line)  # 去掉数字，字母
            sentences += line

    sentence_list = re.split(r'[。]', sentences) #根据句号分割句子，获得句子的list
    # print(sentence_list)


    # data_utils.download_data_gdown("./") # gdrive-ckip # 下载模型
    # 使用 GPU：
    #    1. 安裝 tensorflow-gpu (請見安裝說明)
    #    2. 設定 CUDA_VISIBLE_DEVICES 環境變數，例如：os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    #    3. 設定 disable_cuda=False，例如：ws = WS("./data", disable_cuda=False)
    # 使用 CPU：
    ws = WS("./data")
    pos = POS("./data")
    ner = NER("./data")

    word_sentence_list = ws(  # 对句子进行分词
        sentence_list,
        # sentence_segmentation = True, # To consider delimiters
        # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
        # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
        # coerce_dictionary = dictionary2, # words in this dictionary are forced
    )
    # print(word_sentence_list)

    print(word_sentence_list[0])
    print(sentence_list[0])

    stop_words = []
    with open("./stop_words.txt",encoding='utf-8') as f:
        stop_words = f.readlines()
        stop_words = [stop_word.rstrip() for stop_word in stop_words]

    sentences = []
    for sentence in word_sentence_list:
        words = []
        for word in sentence:
            if word not in stop_words or not word:
                words.append(word)
        sentences.append(' '.join(words))
    sentences

    topic_model = BERTopic(embedding_model=model, language="multilingual", calculate_probabilities=True, verbose=True, nr_topics=10)
    topics, probs = topic_model.fit_transform(sentences)

    print(topic_model.get_topic_info())

    topic_model.get_topic(0)

    fig = topic_model.visualize_barchart()
    fig.write_html(base_path+i[0:4]+"_barchart.html")

    hm = topic_model.visualize_heatmap()
    hm.write_html(base_path+i[0:4]+"_heatmap.html")

['"', '日', '九月', '十年', '○五九一路', '一號', '三樓', '電話鐘', '路餐', '臺灣', '電話', '.', '.', '三四叁式化', '天文臺', '本報', '今日', '復刊', ',', '隨', '報', '附送', '書紙', '精印', '韓國', '地圖', '一', '大', '張', ',', '不', '另', '收費', ',', '請', '讀者', '注意']
"日九月十年○五九一路一號三樓電話鐘路餐臺灣電話..三四叁式化天文臺本報今日復刊,隨報附送書紙精印韓國地圖一大張,不另收費,請讀者注意


Batches:   0%|          | 0/318 [00:00<?, ?it/s]

2022-10-26 11:17:30,368 - BERTopic - Transformed documents to Embeddings
2022-10-26 11:17:35,987 - BERTopic - Reduced dimensionality
2022-10-26 11:17:44,931 - BERTopic - Clustered reduced embeddings
2022-10-26 11:17:47,269 - BERTopic - Reduced number of topics from 137 to 11


    Topic  Count             Name
0      -1   6740   -1_中共_美國_台灣_中國
1       0   1855    0_中共_聯軍_軍事_美國
2       1    269    1_山水_人頭_橋頭_天橋
3       2    254   2_電話_公司_二十年_中共
4       3    217    3_公司_輪船_電話_香港
5       4    175    4_精神_心理_人心_變態
6       5    156    5_上海_人海_海藏_戰術
7       6    155   6_作風_秋風_風流_沁園春
8       7    123    7_汽車_政府_日本_新聞
9       8    105  8_司令_總司令_軍事_司令員
10      9    101    9_女人_野獸_太太_男子
['"', '二', '期', '星臺', ' 文', ' ', '天下', '非', '作', '共', '貨膨', ',', '如', '擊速', '入時軍', ',', '尤以', ',', '勢宾', '否', ',', '尋邏', '約', '噸', '山月', '十', '頭', ',', ' ', '請', '一', '雖', '只', '工足', ',', '所', '戰', '軍幣', '響能國', ',', '戰海', '間', '能', '胥川争 ', '之', '不能', '當頭', '一', ',', '之', '底', '-', '堡聖', '已', '變於', ',', '有理爲長', '亦', '以', '不', ',', '否家', '不', '速', '中', ',', '如爲兩取', ' ', '下', '敵', '而', '於']
"二期星臺 文 天下非作共貨膨,如擊速入時軍,尤以,勢宾否,尋邏約噸山月十頭, 請一雖只工足,所戰軍幣響能國,戰海間能胥川争 之不能當頭一,之底-堡聖已變於,有理爲長亦以不,否家不速中,如爲兩取 下敵而於


Batches:   0%|          | 0/1371 [00:00<?, ?it/s]

...

2022-10-26 23:43:53,600 - BERTopic - Transformed documents to Embeddings
2022-10-26 23:44:09,270 - BERTopic - Reduced dimensionality
2022-10-26 23:46:22,388 - BERTopic - Clustered reduced embeddings
2022-10-26 23:46:28,939 - BERTopic - Reduced number of topics from 265 to 11


    Topic  Count              Name
0      -1  22330    -1_美國_政府_中共_社會
1       0   2096     0_軍事_美國_武器_蘇俄
2       1    776    1_香港_中共_英國_香港人
3       2    740    2_公司_司法_司馬遷_美國
4       3    538             3____
5       4    514     4_汽車_公車_交通_票價
6       5    470     5_風氣_作風_中共_風流
7       6    469     6_水準_江南_水雷_長江
8       7    354     7_金融_資金_金錢_經濟
9       8    334  8_六號_三十七_中華民國_公限
10      9    312    9_美元_定價_四十五_二百
['"-   日', '二月', '一年', '四十七', '國', '民華', '中中華民國', '行政院', '新聞局', '出', ' 業', '登記證局', '僑', '台', '報字', '第○○八', '號', '顧守', '之', '澄清', '洩題', '謠傳', '中華民國', '僑務', '委員會', '登記證', '台教', '新', '字', '第六一二', '號', '主持', '國家', ' 題', '』', '證據']
"-   日二月一年四十七國民華中中華民國行政院新聞局出 業登記證局僑台報字第○○八號顧守之澄清洩題謠傳中華民國僑務委員會登記證台教新字第六一二號主持國家 題』證據


Batches:   0%|          | 0/168 [00:00<?, ?it/s]

2022-10-26 23:49:11,741 - BERTopic - Transformed documents to Embeddings
2022-10-26 23:49:17,885 - BERTopic - Reduced dimensionality
2022-10-26 23:49:20,007 - BERTopic - Clustered reduced embeddings
2022-10-26 23:49:21,661 - BERTopic - Reduced number of topics from 96 to 11


    Topic  Count              Name
0      -1   3764    -1_政府_中共_中國_台灣
1       0    413     0_香港_中共_前途_港人
2       1    350     1_美國_軍隊_軍人_軍事
3       2    141     2_社會_風氣_政府_色情
4       3    138  3_中華民國_四十七_六號_民華
5       4    124     4_漢武_皇帝_皇后_霍光
6       5     93     5_招標_公告_報紙_公開
7       6     88    6_公司_財政司_辭職_評估
8       7     87     7_美國_談判_核武_武器
9       8     85     8_國家_國際_我國_國中
10      9     81             9____


In [26]:
fig.write_html(base_path+year[0:4]+"_barchart.html")

In [28]:
hm.write_html(base_path+year[0:4]+"_heatmap.html")